<a href="https://colab.research.google.com/github/raulFuzita/cct-cloud-services/blob/cloudformation/script_examples/vpc_and_ec2_inst/cloudformation_vpc_and_ec2_instance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CloudFormation Simple Template Generator

Author: Raul Macedo Fuzita

Github: https://github.com/raulFuzita/cct-cloud-services/tree/cloudformation

Email: raul.fuzita@gmail.com

Please, contact me if you have any issue.

## How to Start

We assume you have turn your S3 bucket to **public**, have a copy of your static website in a **zip** format in your bucket, and you have made your zip file public to ACL. If your S3 bucket doesn't meet any of these requirements, scroll down the page to the section **How to Make S3 Bucket Public**. Then come back to this section and read the following instructons.

In [4]:
# Author: Raul Macedo Fuzita
#@markdown 👈 Please, run this cell to install and import all the necessary packages, and resources <font color="red">(required)</font>.

#@markdown <font color="lightblue">After running this cell, you will be asked to restart the runtime again. Please, run this cell again.<font>

import json
import yaml
import requests
import re

!pip install validators
import validators

# https://pypi.org/project/cfn-flip/
!pip install cfn-flip
from cfn_flip import flip, to_yaml, to_json

from google.colab import files

url = "https://raw.githubusercontent.com/raulFuzita/cct-cloud-services/cloudformation/script_examples/vpc_and_ec2_inst/sample_vpc_ec2_inst.json"
r = requests.get(url)
text = r.text.strip()

data = json.loads(text)

## Generate a Template

You can find the default configuration for VPC, Internet Gateway, Route Table, Public Subnet, Security Group, and EC2 Instance in the section **Infrastructure Information**. You're not require to change any configuration.

*   **Description** <font color="lightgreen">(optional)</font>: If you don't provide a description a default one will be added.
    *    Only characters acepted are `:\s._A-Za-z0-9` in regular.
*   **Zip file from S3** <font color="red">(required)</font>: If you don't provide a S3 URL that contains a **zip** file it won't except it.
    *    Example of acept S3 URL: https://yourbucket.s3.amazonaws.com/your_website.zip
*   **Extension** <font color="lightgreen">(optional)</font>: The default format is JSON.
    *    Options: JSON and YAML


In [13]:
# Author: Raul Macedo Fuzita
#@markdown 👈 Please, run this cell after you fill out the fields <font color="red">(required)</font>.<br>

Description = "Test Raul" #@param {type:"string"}
Zip_file_from_S3 = "https://clouds-are-us-raul.s3.amazonaws.com/ca_website.zip" #@param {type:"string"}
Extension = "yml" #@param ["json", "yml"]

if validators.url(Zip_file_from_S3):
    lastslash = Zip_file_from_S3.rindex('/')
    zipfile = Zip_file_from_S3[lastslash+1:]

    part1 = "#!/bin/bash -xe\nyum update -y aws-cfn-bootstrap\n# Install the files and packages from the metadata\n/opt/aws/bin/cfn-init -v --stack ${AWS::StackName} --resource WebServerInstance --configsets All --region ${AWS::Region}\n"
    part2 = "# Download Lab files\nwget " + Zip_file_from_S3 + "\nunzip " + zipfile + " -d /var/www/html/\n"
    part3 = "# Signal the status from cfn-init\n/opt/aws/bin/cfn-signal -e $? --stack ${AWS::StackName} --resource WebServerInstance --region ${AWS::Region}\n"

    userdata = part1 + part2 + part3

    if Description: data['Description'] = re.sub('[^:\s._A-Za-z0-9]+','', Description)
    if zipfile.endswith('.zip'): 
        data['Resources']['WebServerInstance']['Properties']['UserData']['Fn::Base64']['Fn::Sub'] = userdata

        jsonfile = json.dumps(data, indent=4)

        content = jsonfile if Extension == 'json' else to_yaml(jsonfile, clean_up=True)
        filename = 'vpc_and_ec2_instance.'+Extension
        
        with open(filename, 'w') as f:
            f.write(content)
        files.download(filename)
    else:
        print("You must select a zip format")
else:
    print("You must enter a valid url from your S3 bucket")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

If you have generated your template successfully, there is no need to read the rest of the article unless you want to.

## How to Make S3 Bucket Public

If you don't have a bucket you can create one by clicking on **Create bucket** and following the options, or search on YouTube how to create a simple S3 bucket.

If you have an S3 bucket, click on your bucket name.

<img src="https://github.com/raulFuzita/cct-cloud-services/blob/cloudformation/images/public_S3/s3-1.png?raw=true" width=80%/>

Go to tab **Permissions**, click on **Edit** button.

<img src="https://github.com/raulFuzita/cct-cloud-services/blob/cloudformation/images/public_S3/s3-2.png?raw=true" width=75%/>

Unmark the option **Block all public access**. <font color="yellow">Remember this is not aiming safety, the purpose is to import a fake static website to an EC2 instance.</font>

<img src="https://github.com/raulFuzita/cct-cloud-services/blob/cloudformation/images/public_S3/s3-3.png?raw=true" width=75%/>

Upload a copy of your static website in a **zip** format (Don't import in **rar** or any other format, but **zip**).

If you have already a zip file or have uploaded one, select it, click on **Actions**, scroll down the list options, and select **Make public via ACL**.

<img src="https://github.com/raulFuzita/cct-cloud-services/blob/cloudformation/images/public_S3/s3-4.png?raw=true" width=80%/>

In the next window click on **Make public**.

<img src="https://github.com/raulFuzita/cct-cloud-services/blob/cloudformation/images/public_S3/s3-5.png?raw=true" width=75%/>

Click on the zip file name to copy its URL for later step.

<img src="https://github.com/raulFuzita/cct-cloud-services/blob/cloudformation/images/public_S3/s3-6.png?raw=true" width=90%/>

In the **Object URL** property, copy the link and paste it into a notepad or anywhere else.

<img src="https://github.com/raulFuzita/cct-cloud-services/blob/cloudformation/images/public_S3/s3-7.png?raw=true" width=90%/>

Now return to the begining of the tutorial.

## Infrastructure Information

This information is available in the script. You can alter any information after you generate the file. You can use any text editor to do so.


VPC
```
Name: VPC
EnableDnsSupport: true
EnableDnsHostnames: true
CidrBlock: 10.0.0.0/16
```

InternetGateway

```
Name: InternetGateway
Attached to the VPC
```

RouteTable
```
Name: PublicRouteTable
VpcId: VPC

# PublicSubnetRouteTableAssociation
SubnetId: PublicSubnet
RouteTableId: PublicRouteTable
```

Route
```
Name: PublicRoute
DependsOn: VPCGatewayAttachment
RouteTableId: PublicRouteTable
DestinationCidrBlock: 0.0.0.0/0
GatewayId: InternetGateway
```

Subnet
```
Name: PublicSubnet
VpcId: VPC
CidrBlock: 10.0.0.0/24
AvailabilityZone: AWS::Region
```

Subnet Network Acl Association

```
Name: PublicSubnetNetworkAclAssociation
SubnetId: PublicSubnet
NetworkAclId: VPC / DefaultNetworkAcl
```

SecurityGroup
```
Name: WebServerSecurityGroup
VpcId: VPC

# SecurityGroupIngress
IpProtocol: tcp
FromPort: 80
ToPort: 80
CidrIp: 0.0.0.0/0
```

EC2 Instance

```
Name: WebServerInstance

#packages
httpd

InstanceType: t2.micro
ImageId: AmazonLinuxAMIID

# NetworkInterfaces
GroupSet: WebServerSecurityGroup
DeleteOnTermination: true
SubnetId: PublicSubnet

UserData: // Will be generated by this app

# DiskVolume
Size: 100
AvailabilityZone: WebServerInstance / AvailabilityZone

# DiskMountPoint
InstanceId: WebServerInstance
VolumeId: DiskVolume
Device: /dev/sdh
```


